In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import patsy as pt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn import svm 
import statsmodels.api as sm

load data in

In [5]:
stock=pd.read_csv("C:/Users/ngaku/Downloads/dow_jones_index(1).data")
stock.head()

,quarter,stock,date,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
0,1,AA,1/7/2011,$15.82,$16.72,$15.78,$16.42,239655616,3.79267,NaN,NaN,$16.71,$15.97,-4.428490,26,0.182704
1,1,AA,1/14/2011,$16.71,$16.71,$15.64,$15.97,242963398,-4.42849,1.380223,239655616.0,$16.19,$15.79,-2.470660,19,0.187852
2,1,AA,1/21/2011,$16.19,$16.38,$15.60,$15.79,138428495,-2.47066,-43.024959,242963398.0,$15.87,$16.13,1.638310,12,0.189994
3,1,AA,1/28/2011,$15.87,$16.63,$15.82,$16.13,151379173,1.63831,9.355500,138428495.0,$16.18,$17.14,5.933250,5,0.185989
4,1,AA,2/4/2011,$16.18,$17.39,$16.18,$17.14,154387761,5.93325,1.987452,151379173.0,$17.33,$17.37,0.230814,97,0.175029


dropped the stock column because its non numeric

In [6]:
stock = stock.drop(columns=['stock'])

changed the date to an integer that counts days since january 7 2011

In [7]:
stock['date'] = pd.to_datetime(stock['date'])
reference_date = pd.Timestamp('2011-01-07')
stock['date'] = (stock['date'] - reference_date).dt.days
stock.head()

,quarter,date,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
0,1,0,$15.82,$16.72,$15.78,$16.42,239655616,3.79267,NaN,NaN,$16.71,$15.97,-4.428490,26,0.182704
1,1,7,$16.71,$16.71,$15.64,$15.97,242963398,-4.42849,1.380223,239655616.0,$16.19,$15.79,-2.470660,19,0.187852
2,1,14,$16.19,$16.38,$15.60,$15.79,138428495,-2.47066,-43.024959,242963398.0,$15.87,$16.13,1.638310,12,0.189994
3,1,21,$15.87,$16.63,$15.82,$16.13,151379173,1.63831,9.355500,138428495.0,$16.18,$17.14,5.933250,5,0.185989
4,1,28,$16.18,$17.39,$16.18,$17.14,154387761,5.93325,1.987452,151379173.0,$17.33,$17.37,0.230814,97,0.175029


removed all the $

In [8]:
columns_with_dollar_sign = ['open', 'high', 'low', 'close', 'next_weeks_open', 'next_weeks_close']
for column in columns_with_dollar_sign:
    stock[column] = stock[column].str.replace('$', '')
stock[columns_with_dollar_sign] = stock[columns_with_dollar_sign].astype(float)
stock.head()

,quarter,date,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
0,1,0,15.82,16.72,15.78,16.42,239655616,3.79267,NaN,NaN,16.71,15.97,-4.428490,26,0.182704
1,1,7,16.71,16.71,15.64,15.97,242963398,-4.42849,1.380223,239655616.0,16.19,15.79,-2.470660,19,0.187852
2,1,14,16.19,16.38,15.60,15.79,138428495,-2.47066,-43.024959,242963398.0,15.87,16.13,1.638310,12,0.189994
3,1,21,15.87,16.63,15.82,16.13,151379173,1.63831,9.355500,138428495.0,16.18,17.14,5.933250,5,0.185989
4,1,28,16.18,17.39,16.18,17.14,154387761,5.93325,1.987452,151379173.0,17.33,17.37,0.230814,97,0.175029


removed all rows with null values

In [9]:
stock = stock.dropna()
stock.head()

,quarter,date,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
1,1,7,16.71,16.71,15.64,15.97,242963398,-4.428490,1.380223,239655616.0,16.19,15.79,-2.470660,19,0.187852
2,1,14,16.19,16.38,15.60,15.79,138428495,-2.470660,-43.024959,242963398.0,15.87,16.13,1.638310,12,0.189994
3,1,21,15.87,16.63,15.82,16.13,151379173,1.638310,9.355500,138428495.0,16.18,17.14,5.933250,5,0.185989
4,1,28,16.18,17.39,16.18,17.14,154387761,5.933250,1.987452,151379173.0,17.33,17.37,0.230814,97,0.175029
5,1,35,17.33,17.48,16.97,17.37,114691279,0.230814,-25.712195,154387761.0,17.39,17.28,-0.632547,90,0.172712


In [10]:
stock.isna().any()

quarter                               False
date                                  False
open                                  False
high                                  False
low                                   False
close                                 False
volume                                False
percent_change_price                  False
percent_change_volume_over_last_wk    False
previous_weeks_volume                 False
next_weeks_open                       False
next_weeks_close                      False
percent_change_next_weeks_price       False
days_to_next_dividend                 False
percent_return_next_dividend          False
dtype: bool

Train test split

In [11]:
q1 = stock[stock['quarter']==1]
q2 = stock[stock['quarter']==2]

In [12]:
X_train=q1.drop(columns=['close'])
y_train=q1['close']

In [13]:
X_test = q2.drop(columns=['close'])
y_test = q2['close']

Linear model

In [14]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
model_pred = linear_model.predict(X_test)

In [15]:
mse = mean_squared_error(y_test, model_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.172870388538391


Decision Tree

In [17]:
from sklearn.tree import DecisionTreeRegressor


tree_model = DecisionTreeRegressor()


tree_model.fit(X_train, y_train)


y_pred_tree = tree_model.predict(X_test)


tree_mse = mean_squared_error(y_test, y_pred_tree)
print(f'Decision Tree MSE: {tree_mse}')


Decision Tree MSE: 2.3170474358974347


In [18]:
from sklearn.svm import SVR


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


svr_model = SVR()


svr_model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred_svr = svr_model.predict(X_test_scaled)


svr_mse = mean_squared_error(y_test, y_pred_svr)
print(f'SVR MSE: {svr_mse}')


SVR MSE: 831.7280833252445
